### Imports and Jupyter setup

In [2]:
import os
import torch
import wandb
import pandas as pd

from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Custom Imports

In [3]:
from fgvc.utils.augmentations import test_transforms
from fgvc.utils.datasets import TrainDataset, get_dataloader
from fgvc.utils.performance import test_loop, test_loop_insights
from fgvc.utils.utils import timer, init_logger, seed_everything, getModel
from fgvc.utils.wandb import update_wandb_run_test_performance

### Load Dataset Metadata

In [5]:
CKPT_LOCATION = '../experiments/baselines/'

train_metadata = pd.read_csv("../metadata/PlantCLEF2018_train_metadata.csv")
val_metadata = pd.read_csv("../metadata/PlantCLEF2018_val_metadata.csv")

PlantCLEF2017_test = pd.read_csv("../metadata/PlantCLEF2017_test_metadata.csv")
PlantCLEF2018_test = pd.read_csv("../metadata/PlantCLEF2018_test_metadata.csv")

print(f'Number of samples in PlantCLEF2017_test: {len(PlantCLEF2017_test)}')
print(f'Number of observations in PlantCLEF2017_test: {len(PlantCLEF2017_test.ObservationId.unique())}')
print(f'Number of samples in PlantCLEF2018_test: {len(PlantCLEF2018_test)}')
print(f'Number of observations in PlantCLEF2018_test: {len(PlantCLEF2018_test.ObservationId.unique())}')

Number of samples in PlantCLEF2017_test: 25170
Number of samples in PlantCLEF2018_test: 6892


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15,16,17,19,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
LOCAL_PATH = '/Data-10T/PlantCLEF/'

if LOCAL_PATH:
    PlantCLEF2017_test['image_path'] = PlantCLEF2017_test['image_path'].apply(lambda x: x.replace('/local/nahouby/Datasets/PlantCLEF/', LOCAL_PATH))
    PlantCLEF2018_test['image_path'] = PlantCLEF2018_test['image_path'].apply(lambda x: x.replace('/local/nahouby/Datasets/PlantCLEF/', LOCAL_PATH))

### Read WandB runs

In [6]:
api = wandb.Api()
entity, project = "picekl", "frontiers-plant-recognition"  # set to your entity and project 
runs = api.runs(entity + "/" + project)
print(f"Founnd {len(runs)} existing runs.")

Founnd 19 existing runs.


### Evaluate All existing runs

In [7]:
for run in runs:
    if len(run.history()) == 101:
        in_progress = False
    else:
        in_progress = True
        
    config = run.config
    if 'upsampled' in config:
        RUN_NAME = f"{config['architecture']}-{config['optimizer']}-{config['scheduler']}-{config['augmentations']}-{config['upsampled']}"    
    else:
        RUN_NAME = f"{config['architecture']}-{config['optimizer']}-{config['scheduler']}-{config['augmentations']}"  
    
    CKPT_PATH = f'{CKPT_LOCATION}/{RUN_NAME}-100E.pth'
    
    if not in_progress and os.path.isfile(CKPT_PATH) and ("PlantCLEF2017 | Test Acc. (img.)" not in run.summary): 
        print(f'The finished {RUN_NAME} was found and will be evaluated.')
        
        seed_everything(config['random_seed'])
        
        # Init Model
        model = getModel(config['architecture'], config['number_of_classes'], pretrained=False)
        model_mean, model_std = list(model.default_cfg['mean']), list(model.default_cfg['std'])
        model.load_state_dict(torch.load(CKPT_PATH))
        model.to(device)
        model.eval()
        
        print(f'Model Loaded and set to Eval mode.')
        print(f'Model testing started.')
        performance_2017 = test_loop(PlantCLEF2017_test, get_dataloader(PlantCLEF2017_test, config, model_mean, model_std), model, device)
        performance_2018 = test_loop(PlantCLEF2018_test, get_dataloader(PlantCLEF2018_test, config, model_mean, model_std), model, device)

        update_wandb_run_test_performance(run, performance_2017, performance_2018)
        
        del model

The finished inception_resnet_v2-SGD-plateau-light was found and will be evaluated.
Model Loaded and set to Eval mode.
Model testing started.


100%|██████████| 431/431 [00:26<00:00, 16.42it/s]


The finished vit_base_patch16_384-SGD-plateau-light was found and will be evaluated.
Model Loaded and set to Eval mode.
Model testing started.


100%|██████████| 431/431 [00:47<00:00,  9.08it/s]
